In [1]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split
#trainset
dataset = pd.read_csv("/data/fjsdata/physionet/eICU-CRD/EMBC2020/trainset.csv",sep=',',index_col=['patientunitstayid']) 
#min-max scale the continous features
ss = MinMaxScaler()
scale_features = ['ph', 'creatinine', 'albumin','diagnosis']
dataset[scale_features] = ss.fit_transform(dataset[scale_features])
X_data = dataset.drop(columns=["actualiculos"], inplace=False)  #feature
y_data = dataset['actualiculos']#label
X_train, X_val, y_train, y_val = train_test_split(X_data, y_data, test_size=0.2, random_state=1)
print ('The shape of trainset is : %d,%d'%(X_train.shape[0],X_train.shape[1]))
print ('The shape of valset is : %d,%d'%(X_val.shape[0],X_val.shape[1]))

#testset
testset = pd.read_csv("/data/fjsdata/physionet/eICU-CRD/EMBC2020/testset.csv",sep=',',index_col=['patientunitstayid'])
testset[scale_features] = ss.fit_transform(testset[scale_features])
X_test = testset.drop(columns=["actualiculos"], inplace=False)  #feature
y_test = testset['actualiculos']#label 
print ('The shape of testset is : %d,%d'%(X_test.shape[0],X_test.shape[1]))

The shape of trainset is : 87190,52
The shape of valset is : 21798,52
The shape of testset is : 27248,52


In [3]:
from sklearn import linear_model
from sklearn.metrics import mean_absolute_error, r2_score, explained_variance_score,mean_squared_error
from math import sqrt
from sklearn.model_selection import GridSearchCV

#1. training model
param_grid = {'fit_intercept':[True,False],'alpha':[0.01,0.05,0.1,0.5]}
clf = linear_model.Lasso(normalize=False,random_state=0) #max_iter
grid_clf = GridSearchCV(clf, param_grid, cv=5)
grid_clf.fit(X_data, y_data.ravel())

#2. valset performance
y_pred_val = grid_clf.predict(X_val)
mae = mean_absolute_error(y_val, y_pred_val)
print("MAE Score of LS+L1 on eICU-CRD valset is :", mae)  
r2 = r2_score(y_val, y_pred_val)
print("R^2 Score of LS+L1 on eICU-CRD valset is :", r2) 
ev = explained_variance_score(y_val, y_pred_val)
print("EV Score of LS+L1 on eICU-CRD valset is :", ev)

#3. testset performance
y_pred_test = grid_clf.predict(X_test)
mae = mean_absolute_error(y_test, y_pred_test)
print("MAE Score of LS+L1 on eICU-CRD testset is :", mae)  
r2 = r2_score(y_test, y_pred_test)
print("R^2 Score of LS+L1 on eICU-CRD testset is :", r2) 
ev = explained_variance_score(y_test, y_pred_test)
print("EV Score of LS+L1 on eICU-CRD testset is :", ev)

MAE Score of LS+L1 on eICU-CRD valset is : 1.9846851549588076
R^2 Score of LS+L1 on eICU-CRD valset is : 0.11381382199829237
EV Score of LS+L1 on eICU-CRD valset is : 0.11382110898999853
MAE Score of LS+L1 on eICU-CRD testset is : 2.0118698041687835
R^2 Score of LS+L1 on eICU-CRD testset is : 0.09112147903444556
EV Score of LS+L1 on eICU-CRD testset is : 0.0911734768295498


In [5]:
from sklearn import linear_model
from sklearn.metrics import mean_absolute_error, r2_score, explained_variance_score,mean_squared_error
from math import sqrt
from sklearn.model_selection import GridSearchCV

#1. training model
param_grid = {'fit_intercept':[True,False],'alpha':[0.01,0.05,0.1,0.5]}
clf = linear_model.Ridge(normalize=False,random_state=0) #max_iter
grid_clf = GridSearchCV(clf, param_grid, cv=5)
grid_clf.fit(X_data, y_data.ravel())

#2. valset performance
y_pred_val = grid_clf.predict(X_val)
mae = mean_absolute_error(y_val, y_pred_val)
print("MAE Score of LS+L2 on eICU-CRD valset is :", mae)  
r2 = r2_score(y_val, y_pred_val)
print("R^2 Score of LS+L2 on eICU-CRD valset is :", r2) 
ev = explained_variance_score(y_val, y_pred_val)
print("EV Score of LS+L2 on eICU-CRD valset is :", ev)

#3. testset performance
y_pred_test = grid_clf.predict(X_test)
mae = mean_absolute_error(y_test, y_pred_test)
print("MAE Score of LS+L2 on eICU-CRD testset is :", mae)  
r2 = r2_score(y_test, y_pred_test)
print("R^2 Score of LS+L2 on eICU-CRD testset is :", r2) 
ev = explained_variance_score(y_test, y_pred_test)
print("EV Score of LS+L2 on eICU-CRD testset is :", ev)

MAE Score of LS+L2 on eICU-CRD valset is : 1.9786537124978056
R^2 Score of LS+L2 on eICU-CRD valset is : 0.11926766341787076
EV Score of LS+L2 on eICU-CRD valset is : 0.11926781619540616
MAE Score of LS+L2 on eICU-CRD testset is : 2.010006093614097
R^2 Score of LS+L2 on eICU-CRD testset is : 0.09309526751210051
EV Score of LS+L2 on eICU-CRD testset is : 0.09312159570824141


In [6]:
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_absolute_error, r2_score, explained_variance_score,mean_squared_error
from math import sqrt
from sklearn.model_selection import GridSearchCV

#1. training model
param_grid = { 'n_estimators': [5, 10, 15, 20], 'max_depth': [10, 20, 30, 50] }
clf = RandomForestRegressor(max_features='sqrt', min_samples_split=110, min_samples_leaf=20, oob_score=False, random_state=0)
grid_clf = GridSearchCV(clf, param_grid, cv=5)
grid_clf.fit(X_data, y_data.ravel())

#2. valset performance
y_pred_val = grid_clf.predict(X_val)
mae = mean_absolute_error(y_val, y_pred_val)
print("MAE Score of RandomForest on eICU-CRD valset is :", mae)  
r2 = r2_score(y_val, y_pred_val)
print("R^2 Score of RandomForest on eICU-CRD valset is :", r2) 
ev = explained_variance_score(y_val, y_pred_val)
print("EV Score of RandomForest on eICU-CRD valset is :", ev)

#3. testset performance
y_pred_test = grid_clf.predict(X_test)
mae = mean_absolute_error(y_test, y_pred_test)
print("MAE Score ofRandomForest on eICU-CRD testset is :", mae)  
r2 = r2_score(y_test, y_pred_test)
print("R^2 Score of RandomForest on eICU-CRD testset is :", r2) 
ev = explained_variance_score(y_test, y_pred_test)
print("EV Score of RandomForest on eICU-CRD testset is :", ev)

MAE Score of RandomForest on eICU-CRD valset is : 1.8528945201717582
R^2 Score of RandomForest on eICU-CRD valset is : 0.21133307430415182
EV Score of RandomForest on eICU-CRD valset is : 0.21133361822793006
MAE Score ofRandomForest on eICU-CRD testset is : 1.9641036038990178
R^2 Score of RandomForest on eICU-CRD testset is : 0.11057098058538672
EV Score of RandomForest on eICU-CRD testset is : 0.11064609995353891


In [2]:
from sklearn.metrics import mean_absolute_error, r2_score, explained_variance_score,mean_squared_error
from math import sqrt
import tensorflow as tf
import sys
from sklearn.preprocessing import MinMaxScaler
import warnings
warnings.filterwarnings("ignore")

#DNN model
class TF_DNNRegressor_eICU:
    def __init__(self, lr=0.001, dim=52, num_class=1, batchSize=1000):
        #global parameters
        self.lr = lr
        self.dim = dim # dimensions of sample
        self.num_class = num_class #output 
        self.hidden_layers = [16, 4]
        #set network structure
        self.add_placeholders()
        self.add_weight()
        self.add_model()
        self.add_loss()
        self.add_optimizer()
        self.init_sess()
        
    def add_placeholders(self):    
        self.X_input = tf.placeholder("float", [None, self.dim])
        self.Y_input = tf.placeholder("float", [None, self.num_class])
        self.keep_prob = tf.placeholder(tf.float32)  
    
    def add_weight(self):
        # Store layers weight & bias
        #init_uniform = tf.random_uniform_initializer(minval=0, maxval=1, seed=None, dtype=tf.float32)
        self.weights = {
            'w1': tf.Variable(tf.random_normal([self.dim, self.hidden_layers[0]])),
            'w2': tf.Variable(tf.random_normal([self.hidden_layers[0], self.hidden_layers[1]])),
            'wout': tf.Variable(tf.random_normal([self.hidden_layers[1], self.num_class]))
        }
        self.biases = {
            'b1': tf.Variable(tf.random_normal([self.hidden_layers[0]])),
            'b2': tf.Variable(tf.random_normal([self.hidden_layers[1]])),
            'bout': tf.Variable(tf.random_normal([self.num_class]))
        }
        
    def add_model(self):
        # Hidden fully connected layer with 16 neurons
        layer_1 =  tf.add(tf.matmul(self.X_input, self.weights['w1']), self.biases['b1']) 
        # Hidden fully connected layer with 4 neurons
        layer_2 = tf.add(tf.matmul(layer_1, self.weights['w2']), self.biases['b2']) 
        # Output fully connected layer with a neuron for each class
        out_layer =tf.matmul(layer_2, self.weights['wout']) + self.biases['bout'] 
        self.Y_output =  out_layer
    
    def add_loss(self):
         self.loss = tf.losses.mean_squared_error( self.Y_input , self.Y_output ) 
            
    def add_optimizer(self):
        optimizer = tf.train.AdamOptimizer(self.lr)
        self.train_step = optimizer.minimize(self.loss)
        
    def init_sess(self):
        self.config = tf.ConfigProto()
        self.config.gpu_options.allow_growth = True
        self.config.allow_soft_placement = True
        #self.config.gpu_options.per_process_gpu_memory_fraction = 0.5
        self.sess = tf.Session(config=self.config)
        self.sess.run(tf.global_variables_initializer())
        
#1.training model
tf_model = TF_DNNRegressor_eICU()
#set paramete
verbose = 10
batchSize=1000
num_batches = X_data.shape[0] // batchSize + 1 
pre_loss = 0.0
while True:#convergence
    losses = []
    for i in range(num_batches):
        min_idx = i * batchSize
        max_idx = np.min([X_data.shape[0], (i+1)*batchSize])
        X_batch = X_data[min_idx: max_idx]
        Y_batch = y_data.to_frame()[min_idx: max_idx]
        _, tmp_loss,  = tf_model.sess.run([tf_model.train_step, tf_model.loss], 
                                         feed_dict={tf_model.X_input: X_batch,tf_model.Y_input: Y_batch, tf_model.keep_prob: 0.6})
        losses.append(tmp_loss)
        if verbose and i % verbose == 0:
            sys.stdout.write('\r{} / {} : loss = {}'.format(i, num_batches, np.mean(losses[-verbose:])))
            sys.stdout.flush()
    sys.stdout.write("\nMean loss in this epoch is: {}".format( np.mean(losses) ))
    sys.stdout.flush()
    #whether convergence
    if abs( np.mean(losses) - pre_loss)<0.001:
        break
    else:
        pre_loss = np.mean(losses)
        
#2. valset performance
y_pred_val = tf_model.sess.run(tf_model.Y_output, feed_dict={tf_model.X_input: X_val,tf_model.Y_input: y_val.to_frame(), tf_model.keep_prob: 1})
mae = mean_absolute_error(y_val, y_pred_val)
print("MAE Score of DeepNN on eICU-CRD valset is :", mae)  
r2 = r2_score(y_val, y_pred_val)
print("R^2 Score of DeepNN on eICU-CRD valset is :", r2) 
ev = explained_variance_score(y_val, y_pred_val)
print("EV Score of DeepNN on eICU-CRD valset is :", ev)

#3. testset performance
y_pred_test = tf_model.sess.run(tf_model.Y_output, feed_dict={tf_model.X_input: X_test,tf_model.Y_input: y_test.to_frame(), tf_model.keep_prob: 1})
mae = mean_absolute_error(y_test, y_pred_test)
print("MAE Score of DeepNN on eICU-CRD testset is :", mae)  
r2 = r2_score(y_test, y_pred_test)
print("R^2 Score of DeepNN on eICU-CRD testset is :", r2) 
ev = explained_variance_score(y_test, y_pred_test)
print("EV Score of DeepNN on eICU-CRD testset is :", ev)

/root/miniconda3/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:516: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/root/miniconda3/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/root/miniconda3/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:518: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/root/miniconda3/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:519: FutureWarnin

Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where


/root/miniconda3/lib/python3.7/site-packages/tensorboard/compat/tensorflow_stub/dtypes.py:541: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/root/miniconda3/lib/python3.7/site-packages/tensorboard/compat/tensorflow_stub/dtypes.py:542: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/root/miniconda3/lib/python3.7/site-packages/tensorboard/compat/tensorflow_stub/dtypes.py:543: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/root/miniconda3/lib/python3.7/site-packages/tensorboard/compat/tensorflow_stub

100 / 109 : loss = 86.83037567138672
100 / 109 : loss = 55.04607391357422978515625
100 / 109 : loss = 41.34906387329101640551758
100 / 109 : loss = 33.08215332031256815551758
100 / 109 : loss = 27.847930908203125979003906
100 / 109 : loss = 24.41364097595215387084961
100 / 109 : loss = 22.08343887329101699609375
100 / 109 : loss = 20.454273223876953533569336
100 / 109 : loss = 19.287708282470703685913086
100 / 109 : loss = 18.43716049194336846435547
100 / 109 : loss = 17.807607650756836494628906
100 / 109 : loss = 17.33424949645996626220703
100 / 109 : loss = 16.971525192260742400634766
100 / 109 : loss = 16.687166213989258704589844
100 / 109 : loss = 16.458526611328125192260742
100 / 109 : loss = 16.269981384277344080505371
100 / 109 : loss = 16.110879898071298012695312
100 / 109 : loss = 15.973997116088867329833984
100 / 109 : loss = 15.854377746582031344970703
100 / 109 : loss = 15.748559951782227836547852
100 / 109 : loss = 15.654035568237305243103027
100 / 109 : loss = 15.56894493

In [10]:
import pandas as pd
import time
import copy
import math
import matplotlib.pyplot as plt
import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.autograd import Variable
from torch.optim import Optimizer
from sklearn.model_selection import KFold
from torchvision import datasets, transforms
from math import sqrt
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import mean_absolute_error, r2_score, explained_variance_score,mean_squared_error
print (torch.cuda.is_available())
print (torch.version.cuda)
print (torch.cuda.get_device_name(torch.cuda.current_device()))
#model
def to_variable(var=(), cuda=True, volatile=False):
    out = []
    for v in var:
        
        if isinstance(v, np.ndarray):
            v = torch.from_numpy(v).type(torch.FloatTensor)

        if not v.is_cuda and cuda:
            v = v.cuda()

        if not isinstance(v, Variable):
            v = Variable(v, volatile=volatile)

        out.append(v)
    return out

def log_gaussian_loss(output, target, sigma, no_dim, sum_reduce=True):
    exponent = -0.5*(target - output)**2/sigma**2
    log_coeff = -no_dim*torch.log(sigma) - 0.5*no_dim*np.log(2*np.pi)
    
    if sum_reduce:
        return -(log_coeff + exponent).sum()
    else:
        return -(log_coeff + exponent)
    
class gaussian:
    def __init__(self, mu, sigma):
        self.mu = mu
        self.sigma = sigma
        
    def loglik(self, weights):
        exponent = -0.5*(weights - self.mu)**2/self.sigma**2
        log_coeff = -0.5*(np.log(2*np.pi) + 2*np.log(self.sigma))
        
        return (exponent + log_coeff).sum()
    
class BayesLinear_Normalq(nn.Module):
    def __init__(self, input_dim, output_dim, prior):
        super(BayesLinear_Normalq, self).__init__()
        self.input_dim = input_dim
        self.output_dim = output_dim
        self.prior = prior
        
        self.weight_mus = nn.Parameter(torch.Tensor(self.input_dim, self.output_dim).uniform_(-0.01, 0.01))
        self.weight_rhos = nn.Parameter(torch.Tensor(self.input_dim, self.output_dim).uniform_(-3, -3))
        
    def forward(self, x):
        # sample gaussian noise for each weight
        weight_epsilons = Variable(self.weight_mus.data.new(self.weight_mus.size()).normal_())      
        # calculate the weight stds from the rho parameters
        weight_stds = torch.log(1 + torch.exp(self.weight_rhos))
        # calculate samples from the posterior from the sampled noise and mus/stds
        weight_sample = self.weight_mus + weight_epsilons*weight_stds
            
        torch.cuda.synchronize()
        output = torch.mm(x, weight_sample)
            
        # computing the KL loss term
        #reference: https://github.com/jojonki/AutoEncoders/blob/master/kl_divergence_between_two_gaussians.pdf
        prior_cov, varpost_cov = self.prior.sigma**2, weight_stds**2
        KL_loss = 0.5*(torch.log(prior_cov/varpost_cov)).sum() - 0.5*weight_stds.numel()
        KL_loss = KL_loss + 0.5*(varpost_cov/prior_cov).sum()
        KL_loss = KL_loss + 0.5*((self.weight_mus - self.prior.mu)**2/prior_cov).sum()
            
        return output, KL_loss
    
class BBP_Model(nn.Module):
    def __init__(self, input_dim, output_dim, num_units):
        super(BBP_Model, self).__init__()
        
        self.input_dim = input_dim
        self.output_dim = output_dim
        
        # network with two hidden and one output layer
        self.layer1 = BayesLinear_Normalq(input_dim, num_units[0], gaussian(0, 3))
        self.layer2 = BayesLinear_Normalq(num_units[0], num_units[1], gaussian(0, 3))
        self.layer3 = BayesLinear_Normalq(num_units[1], output_dim, gaussian(0, 3))
        
        # activation to be used between hidden layers
        self.activation = nn.ReLU(inplace = True)
        # noise
        self.log_noise = nn.Parameter(torch.cuda.FloatTensor([3]))
    
    def forward(self, x):
        
        KL_loss_total = 0
        x = x.view(-1, self.input_dim)
        
        x, KL_loss = self.layer1(x)
        KL_loss_total = KL_loss_total + KL_loss
        x = self.activation(x)
        
        x, KL_loss = self.layer2(x)
        KL_loss_total = KL_loss_total + KL_loss
        x = self.activation(x)
        
        x, KL_loss = self.layer3(x)
        KL_loss_total = KL_loss_total + KL_loss
        x = self.activation(x)
        
        return x, KL_loss_total
    
class BBP_Model_Wrapper:
    def __init__(self, network, learn_rate=1e-2):
        
        self.learn_rate = learn_rate
        self.network = network
        self.network.cuda()
        
        self.optimizer = torch.optim.Adam(self.network.parameters(), lr = self.learn_rate)
        self.loss_func = log_gaussian_loss#nn.MSELoss() 
    
    def fit(self, x, y, no_samples):
        
        x, y = to_variable(var=(x, y), cuda=True)
        
        # reset gradient and total loss
        self.optimizer.zero_grad()
        fit_loss_total = 0
        KL_loss_total = 0
        for i in range(no_samples):
            output, KL_loss = self.network(x)
            KL_loss_total = KL_loss_total + KL_loss
            # calculate fit loss based on mean and standard deviation of output
            fit_loss = self.loss_func(output, y, self.network.log_noise.exp(), 1) 
            fit_loss_total = fit_loss_total + fit_loss
        
        total_loss = (fit_loss_total + KL_loss_total)/(no_samples*x.shape[0])
        total_loss.backward()
        self.optimizer.step()

        return total_loss
    
#1.training model
num_epochs = 300
log_every=10
best_net, best_loss = None, float('inf')
net = BBP_Model_Wrapper(network=BBP_Model(input_dim=X_data.shape[1], output_dim=1, num_units=[128,32]))
for i in range(num_epochs):
    total_loss = net.fit(np.array(X_data), np.array(y_data.to_frame()), no_samples = 100)

    if total_loss < best_loss:
        best_loss = total_loss
        best_net = copy.deepcopy(net.network)
            
    if i % log_every == 0 or i == num_epochs - 1:
        print('Epoch: %s/%d, loss = %.3f' %(str(i).zfill(3), num_epochs, total_loss))
        
print ('The best loss = %.3f'%(best_loss))
        
#2. valset performance
X_val_cuda, y_val_cuda = to_variable(var=(np.array(X_val), np.array(y_val.to_frame())), cuda=True)
y_pred_val= []
for i in range(200):#sample
    output, KL_loss = best_net(X_val_cuda)
    y_pred_val.append(output.cpu().data.numpy())
y_pred_val = np.array(y_pred_val).mean(axis=0)    
mae=mean_absolute_error(y_val, y_pred_val)
r2=r2_score(y_val, y_pred_val)
ev=explained_variance_score(y_val, y_pred_val)
print("MAE Score of BNN on eICU-CRD valset is :", mae)    
print("R^2 Score of BNN on eICU-CRD valset is :", r2)  
print("EV Score of BNN on eICU-CRD valset is :", ev) 

#3. testset performance
X_test_cuda, y_test_cuda = to_variable(var=(np.array(X_test), np.array(y_test.to_frame())), cuda=True)
y_test_val= []
for i in range(200):#sample
    output, KL_loss = best_net(X_test_cuda)
    y_test_val.append(output.cpu().data.numpy())
y_test_val = np.array(y_test_val).mean(axis=0)  
mae=mean_absolute_error(y_test, y_test_val)
r2=r2_score(y_test, y_test_val)
ev=explained_variance_score(y_test, y_test_val)
print("MAE Score of BNN on eICU-CRD testset is :", mae)    
print("R^2 Score of BNN on eICU-CRD testset is :", r2)  
print("EV Score of BNN on eICU-CRD testset is :", ev) 

True
9.0.176
GeForce RTX 2080 Ti
Epoch: 000/300, loss = 4.199
Epoch: 010/300, loss = 4.082
Epoch: 020/300, loss = 3.977
Epoch: 030/300, loss = 3.873
Epoch: 040/300, loss = 3.772
Epoch: 050/300, loss = 3.672
Epoch: 060/300, loss = 3.575
Epoch: 070/300, loss = 3.482
Epoch: 080/300, loss = 3.393
Epoch: 090/300, loss = 3.309
Epoch: 100/300, loss = 3.230
Epoch: 110/300, loss = 3.157
Epoch: 120/300, loss = 3.092
Epoch: 130/300, loss = 3.034
Epoch: 140/300, loss = 2.985
Epoch: 150/300, loss = 2.943
Epoch: 160/300, loss = 2.911
Epoch: 170/300, loss = 2.884
Epoch: 180/300, loss = 2.864
Epoch: 190/300, loss = 2.850
Epoch: 200/300, loss = 2.838
Epoch: 210/300, loss = 2.829
Epoch: 220/300, loss = 2.820
Epoch: 230/300, loss = 2.815
Epoch: 240/300, loss = 2.808
Epoch: 250/300, loss = 2.804
Epoch: 260/300, loss = 2.799
Epoch: 270/300, loss = 2.796
Epoch: 280/300, loss = 2.791
Epoch: 290/300, loss = 2.790
Epoch: 299/300, loss = 2.785
The best loss = 2.785
MAE Score of BNN on eICU-CRD valset is : 1.932

In [ ]:
import pandas as pd
import time
import copy
import math
import matplotlib.pyplot as plt
import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.autograd import Variable
from torch.optim import Optimizer
from sklearn.model_selection import KFold
from torchvision import datasets, transforms
from math import sqrt
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import mean_absolute_error, r2_score, explained_variance_score,mean_squared_error
print (torch.cuda.is_available())
print (torch.version.cuda)
print (torch.cuda.get_device_name(torch.cuda.current_device()))
#model
def to_variable(var=(), cuda=True, volatile=False):
    out = []
    for v in var:
        
        if isinstance(v, np.ndarray):
            v = torch.from_numpy(v).type(torch.FloatTensor)

        if not v.is_cuda and cuda:
            v = v.cuda()

        if not isinstance(v, Variable):
            v = Variable(v, volatile=volatile)

        out.append(v)
    return out

def log_gaussian_loss(output, target, sigma, no_dim, sum_reduce=True):
    exponent = -0.5*(target - output)**2/sigma**2
    log_coeff = -no_dim*torch.log(sigma) - 0.5*no_dim*np.log(2*np.pi)
    
    if sum_reduce:
        return -(log_coeff + exponent).sum()
    else:
        return -(log_coeff + exponent)
    
class gaussian:
    def __init__(self, mu, sigma):
        self.mu = mu
        self.sigma = sigma
        
    def loglik(self, weights):
        exponent = -0.5*(weights - self.mu)**2/self.sigma**2
        log_coeff = -0.5*(np.log(2*np.pi) + 2*np.log(self.sigma))
        
        return (exponent + log_coeff).sum()
    
class BayesLinear_Normalq(nn.Module):
    def __init__(self, input_dim, output_dim, prior):
        super(BayesLinear_Normalq, self).__init__()
        self.input_dim = input_dim
        self.output_dim = output_dim
        self.prior = prior
        
        self.weight_mus = nn.Parameter(torch.Tensor(self.input_dim, self.output_dim).uniform_(-0.01, 0.01))
        self.weight_rhos = nn.Parameter(torch.Tensor(self.input_dim, self.output_dim).uniform_(-3, -3))
        
    def forward(self, x):
        # sample gaussian noise for each weight
        weight_epsilons = Variable(self.weight_mus.data.new(self.weight_mus.size()).normal_())      
        # calculate the weight stds from the rho parameters
        weight_stds = torch.log(1 + torch.exp(self.weight_rhos))
        # calculate samples from the posterior from the sampled noise and mus/stds
        weight_sample = self.weight_mus + weight_epsilons*weight_stds
            
        torch.cuda.synchronize()
        output = torch.mm(x, weight_sample)
            
        # computing the KL loss term
        #reference: https://github.com/jojonki/AutoEncoders/blob/master/kl_divergence_between_two_gaussians.pdf
        prior_cov, varpost_cov = self.prior.sigma**2, weight_stds**2
        KL_loss = 0.5*(torch.log(prior_cov/varpost_cov)).sum() - 0.5*weight_stds.numel()
        KL_loss = KL_loss + 0.5*(varpost_cov/prior_cov).sum()
        KL_loss = KL_loss + 0.5*((self.weight_mus - self.prior.mu)**2/prior_cov).sum()
            
        return output, KL_loss
    
class BBP_Model(nn.Module):
    def __init__(self, input_dim, output_dim, num_units):
        super(BBP_Model, self).__init__()
        
        self.input_dim = input_dim
        self.output_dim = output_dim
        
        # network with two hidden and one output layer
        self.layer1 = BayesLinear_Normalq(input_dim, num_units[0], gaussian(0, 3))
        self.layer2 = BayesLinear_Normalq(num_units[0], num_units[1], gaussian(0, 3))
        self.layer3 = BayesLinear_Normalq(num_units[1], output_dim, gaussian(0, 3))
        
        # activation to be used between hidden layers
        self.activation = nn.ReLU(inplace = True)
        # noise
        self.log_noise = nn.Parameter(torch.cuda.FloatTensor([3]))
    
    def forward(self, x):
        
        KL_loss_total = 0
        x = x.view(-1, self.input_dim)
        
        x, KL_loss = self.layer1(x)
        KL_loss_total = KL_loss_total + KL_loss
        x = self.activation(x)
        
        x, KL_loss = self.layer2(x)
        KL_loss_total = KL_loss_total + KL_loss
        x = self.activation(x)
        
        x, KL_loss = self.layer3(x)
        KL_loss_total = KL_loss_total + KL_loss
        x = self.activation(x)
        
        return x, KL_loss_total
    
class BBP_Model_Wrapper:
    def __init__(self, network, learn_rate=1e-2):
        
        self.learn_rate = learn_rate
        self.network = network
        self.network.cuda()
        
        self.optimizer = torch.optim.Adam(self.network.parameters(), lr = self.learn_rate)
        self.loss_func = log_gaussian_loss#nn.MSELoss() 
    
    def fit(self, x, y, no_samples):
        
        x, y = to_variable(var=(x, y), cuda=True)
        
        # reset gradient and total loss
        self.optimizer.zero_grad()
        fit_loss_total = 0
        KL_loss_total = 0
        for i in range(no_samples):
            output, KL_loss = self.network(x)
            KL_loss_total = KL_loss_total + KL_loss
            # calculate fit loss based on mean and standard deviation of output
            fit_loss = self.loss_func(output, y, self.network.log_noise.exp(), 1) 
            fit_loss_total = fit_loss_total + fit_loss
        
        total_loss = (fit_loss_total + KL_loss_total)/(no_samples*x.shape[0])
        total_loss.backward()
        self.optimizer.step()

        return total_loss
    
#1.training model
num_epochs = 500
log_every=10
best_net, best_loss = None, float('inf')
net = BBP_Model_Wrapper(network=BBP_Model(input_dim=X_data.shape[1], output_dim=1, num_units=[128,32]))
for i in range(num_epochs):
    total_loss = net.fit(np.array(X_data), np.array(y_data.to_frame()), no_samples = 100)

    if total_loss < best_loss:
        best_loss = total_loss
        best_net = copy.deepcopy(net.network)
            
    if i % log_every == 0 or i == num_epochs - 1:
        print('Epoch: %s/%d, loss = %.3f' %(str(i).zfill(3), num_epochs, total_loss))
        
print ('The best loss = %.3f'%(best_loss))
        
#2. valset performance
X_val_cuda, y_val_cuda = to_variable(var=(np.array(X_val), np.array(y_val.to_frame())), cuda=True)
y_pred_val= []
for i in range(200):#sample
    output, KL_loss = best_net(X_val_cuda)
    y_pred_val.append(output.cpu().data.numpy())
y_pred_val = np.array(y_pred_val).mean(axis=0)    
mae=mean_absolute_error(y_val, y_pred_val)
r2=r2_score(y_val, y_pred_val)
ev=explained_variance_score(y_val, y_pred_val)
print("MAE Score of BNN on eICU-CRD valset is :", mae)    
print("R^2 Score of BNN on eICU-CRD valset is :", r2)  
print("EV Score of BNN on eICU-CRD valset is :", ev) 

#3. testset performance
X_test_cuda, y_test_cuda = to_variable(var=(np.array(X_test), np.array(y_test.to_frame())), cuda=True)
y_test_val= []
for i in range(200):#sample
    output, KL_loss = best_net(X_test_cuda)
    y_test_val.append(output.cpu().data.numpy())
y_test_val = np.array(y_test_val).mean(axis=0)  
mae=mean_absolute_error(y_test, y_test_val)
r2=r2_score(y_test, y_test_val)
ev=explained_variance_score(y_test, y_test_val)
print("MAE Score of BNN on eICU-CRD testset is :", mae)    
print("R^2 Score of BNN on eICU-CRD testset is :", r2)  
print("EV Score of BNN on eICU-CRD testset is :", ev) 

True
9.0.176
GeForce RTX 2080 Ti
Epoch: 000/500, loss = 4.308
Epoch: 010/500, loss = 4.191
Epoch: 020/500, loss = 4.087
Epoch: 030/500, loss = 3.982
Epoch: 040/500, loss = 3.880
Epoch: 050/500, loss = 3.781
Epoch: 060/500, loss = 3.684
Epoch: 070/500, loss = 3.590
Epoch: 080/500, loss = 3.501
Epoch: 090/500, loss = 3.416
Epoch: 100/500, loss = 3.338
Epoch: 110/500, loss = 3.264
Epoch: 120/500, loss = 3.199
Epoch: 130/500, loss = 3.140
Epoch: 140/500, loss = 3.091
Epoch: 150/500, loss = 3.050
Epoch: 160/500, loss = 3.014
Epoch: 170/500, loss = 2.988
Epoch: 180/500, loss = 2.967
Epoch: 190/500, loss = 2.951
Epoch: 200/500, loss = 2.939
Epoch: 210/500, loss = 2.928
Epoch: 220/500, loss = 2.923
Epoch: 230/500, loss = 2.913
Epoch: 240/500, loss = 2.907
Epoch: 250/500, loss = 2.901
Epoch: 260/500, loss = 2.894
Epoch: 270/500, loss = 2.889
Epoch: 280/500, loss = 2.882
Epoch: 290/500, loss = 2.879
Epoch: 300/500, loss = 2.873
Epoch: 310/500, loss = 2.869
Epoch: 320/500, loss = 2.867
Epoch: 330

In [6]:
import pandas as pd
import time
import copy
import math
import matplotlib.pyplot as plt
import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.autograd import Variable
from torch.optim import Optimizer
from sklearn.model_selection import KFold
from torchvision import datasets, transforms
from math import sqrt
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import mean_absolute_error, r2_score, explained_variance_score,mean_squared_error
print (torch.cuda.is_available())
print (torch.version.cuda)
print (torch.cuda.get_device_name(torch.cuda.current_device()))
#model
def to_variable(var=(), cuda=True, volatile=False):
    out = []
    for v in var:
        
        if isinstance(v, np.ndarray):
            v = torch.from_numpy(v).type(torch.FloatTensor)

        if not v.is_cuda and cuda:
            v = v.cuda()

        if not isinstance(v, Variable):
            v = Variable(v, volatile=volatile)

        out.append(v)
    return out

def log_gaussian_loss(output, target, sigma, no_dim, sum_reduce=True):
    exponent = -0.5*(target - output)**2/sigma**2
    log_coeff = -no_dim*torch.log(sigma) - 0.5*no_dim*np.log(2*np.pi)
    
    if sum_reduce:
        return -(log_coeff + exponent).sum()
    else:
        return -(log_coeff + exponent)
    
class gaussian:
    def __init__(self, mu, sigma):
        self.mu = mu
        self.sigma = sigma
        
    def loglik(self, weights):
        exponent = -0.5*(weights - self.mu)**2/self.sigma**2
        log_coeff = -0.5*(np.log(2*np.pi) + 2*np.log(self.sigma))
        
        return (exponent + log_coeff).sum()
    
class BayesLinear_Normalq(nn.Module):
    def __init__(self, input_dim, output_dim, prior):
        super(BayesLinear_Normalq, self).__init__()
        self.input_dim = input_dim
        self.output_dim = output_dim
        self.prior = prior
        
        self.weight_mus = nn.Parameter(torch.Tensor(self.input_dim, self.output_dim).uniform_(-0.01, 0.01))
        self.weight_rhos = nn.Parameter(torch.Tensor(self.input_dim, self.output_dim).uniform_(-3, -3))
        
    def forward(self, x):
        # sample gaussian noise for each weight
        weight_epsilons = Variable(self.weight_mus.data.new(self.weight_mus.size()).normal_())      
        # calculate the weight stds from the rho parameters
        weight_stds = torch.log(1 + torch.exp(self.weight_rhos))
        # calculate samples from the posterior from the sampled noise and mus/stds
        weight_sample = self.weight_mus + weight_epsilons*weight_stds
            
        #torch.cuda.synchronize()
        output = torch.mm(x, weight_sample)
            
        # computing the KL loss term
        #reference: https://github.com/jojonki/AutoEncoders/blob/master/kl_divergence_between_two_gaussians.pdf
        prior_cov, varpost_cov = self.prior.sigma**2, weight_stds**2
        KL_loss = 0.5*(torch.log(prior_cov/varpost_cov)).sum() - 0.5*weight_stds.numel()
        KL_loss = KL_loss + 0.5*(varpost_cov/prior_cov).sum()
        KL_loss = KL_loss + 0.5*((self.weight_mus - self.prior.mu)**2/prior_cov).sum()
            
        return output, KL_loss
    
class BBP_Model(nn.Module):
    def __init__(self, input_dim, output_dim, num_units):
        super(BBP_Model, self).__init__()
        
        self.input_dim = input_dim
        self.output_dim = output_dim
        # network with two hidden and one output layer
        self.layer1 = BayesLinear_Normalq(input_dim, num_units[0], gaussian(0, 1))
        self.layer2 = BayesLinear_Normalq(num_units[0], num_units[1], gaussian(0, 1))
        self.layer3 = BayesLinear_Normalq(num_units[1], output_dim, gaussian(0, 1))
        
        # activation to be used between hidden layers
        self.activation = nn.ReLU(inplace = True)
        # noise
        self.log_noise = nn.Parameter(torch.cuda.FloatTensor([3]))
    
    def forward(self, x):
        
        KL_loss_total = 0
        x = x.view(-1, self.input_dim)
        
        x, KL_loss = self.layer1(x)
        KL_loss_total = KL_loss_total + KL_loss
        x = self.activation(x)
        
        x, KL_loss = self.layer2(x)
        KL_loss_total = KL_loss_total + KL_loss
        x = self.activation(x)
        
        x, KL_loss = self.layer3(x)
        KL_loss_total = KL_loss_total + KL_loss
        x = self.activation(x)
        
        return x, KL_loss_total
    
class BBP_Model_Wrapper:
    def __init__(self, network, learn_rate=1e-2):
        
        self.learn_rate = learn_rate
        self.loss_func = log_gaussian_loss#nn.MSELoss() 
        
        self.network = network
        self.network.cuda()
        self.network = nn.DataParallel(self.network,device_ids=[0,1,2,3,4,5,6,7])
        
        self.optimizer = torch.optim.Adam(self.network.parameters(), lr = self.learn_rate)
        self.optimizer = nn.DataParallel(self.optimizer,device_ids=[0,1,2,3,4,5,6,7])
    
    def fit(self, x, y, no_samples):
        
        x, y = to_variable(var=(x, y), cuda=True)
        
        # reset gradient and total loss
        self.optimizer.zero_grad()
        fit_loss_total = 0
        KL_loss_total = 0
        for i in range(no_samples):
            output, KL_loss = self.network(x)
            KL_loss_total = KL_loss_total + KL_loss
            # calculate fit loss based on mean and standard deviation of output
            fit_loss = self.loss_func(output, y, self.network.module.log_noise.exp(), 1) 
            fit_loss_total = fit_loss_total + fit_loss
        
        total_loss = (fit_loss_total + KL_loss_total)/(no_samples*x.shape[0])
        total_loss.mean().backward()#mean()
        self.optimizer.module.step()

        return total_loss
    
#1.training model
num_epochs = 200
log_every=10
best_net, best_loss = None, float('inf')
net = BBP_Model_Wrapper(network=BBP_Model(input_dim=X_data.shape[1], output_dim=1, num_units=[256,64])) #128,32
for i in range(num_epochs):
    total_loss = net.fit(np.array(X_data), np.array(y_data.to_frame()), no_samples = 100).mean()

    if total_loss < best_loss:
        best_loss = total_loss
        best_net = copy.deepcopy(net.network)
            
    if i % log_every == 0 or i == num_epochs - 1:
        print('Epoch: %s/%d, loss = %.3f' %(str(i).zfill(3), num_epochs, total_loss))
        
print ('The best loss = %.3f'%(best_loss))
        
#2. valset performance
X_val_cuda, y_val_cuda = to_variable(var=(np.array(X_val), np.array(y_val.to_frame())), cuda=True)
y_pred_val= []
for i in range(200):#sample
    output, KL_loss = best_net(X_val_cuda)
    y_pred_val.append(output.cpu().data.numpy())
y_pred_val = np.array(y_pred_val).mean(axis=0)    
mae=mean_absolute_error(y_val, y_pred_val)
r2=r2_score(y_val, y_pred_val)
ev=explained_variance_score(y_val, y_pred_val)
print("MAE Score of BNN on eICU-CRD valset is :", mae)    
print("R^2 Score of BNN on eICU-CRD valset is :", r2)  
print("EV Score of BNN on eICU-CRD valset is :", ev) 

#3. testset performance
X_test_cuda, y_test_cuda = to_variable(var=(np.array(X_test), np.array(y_test.to_frame())), cuda=True)
y_test_val= []
for i in range(200):#sample
    output, KL_loss = best_net(X_test_cuda)
    y_test_val.append(output.cpu().data.numpy())
y_test_val = np.array(y_test_val).mean(axis=0)  
mae=mean_absolute_error(y_test, y_test_val)
r2=r2_score(y_test, y_test_val)
ev=explained_variance_score(y_test, y_test_val)
print("MAE Score of BNN on eICU-CRD testset is :", mae)    
print("R^2 Score of BNN on eICU-CRD testset is :", r2)  
print("EV Score of BNN on eICU-CRD testset is :", ev) 

True
9.0.176
GeForce RTX 2080 Ti


/root/miniconda3/lib/python3.7/site-packages/torch/nn/parallel/_functions.py:61: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn('Was asked to gather along dimension 0, but all '


Epoch: 000/200, loss = 4.639
Epoch: 010/200, loss = 4.520
Epoch: 020/200, loss = 4.394
Epoch: 030/200, loss = 4.257
Epoch: 040/200, loss = 4.117
Epoch: 050/200, loss = 3.975
Epoch: 060/200, loss = 3.837
Epoch: 070/200, loss = 3.707
Epoch: 080/200, loss = 3.589
Epoch: 090/200, loss = 3.490
Epoch: 100/200, loss = 3.416
Epoch: 110/200, loss = 3.376
Epoch: 120/200, loss = 3.377
Epoch: 130/200, loss = 3.427
Epoch: 140/200, loss = 3.525
Epoch: 150/200, loss = 3.657
Epoch: 160/200, loss = 3.787
Epoch: 170/200, loss = 3.856
Epoch: 180/200, loss = 3.810
Epoch: 190/200, loss = 3.646
Epoch: 199/200, loss = 3.450
The best loss = 3.371
MAE Score of BNN on eICU-CRD valset is : 2.929296156678186
R^2 Score of BNN on eICU-CRD valset is : -0.5817041778406427
EV Score of BNN on eICU-CRD valset is : -4.513833751218499e-10


/root/miniconda3/lib/python3.7/site-packages/torch/nn/parallel/_functions.py:61: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn('Was asked to gather along dimension 0, but all '


MAE Score of BNN on eICU-CRD testset is : 2.9506505944370716
R^2 Score of BNN on eICU-CRD testset is : -0.4412636601685773
EV Score of BNN on eICU-CRD testset is : 3.4406244076023995e-08
